# This code is to scrape hosuing information from rew.ca website

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

# Base directory to save CSV files
base_dir = "C:\\Users\\mahagam3\\Documents\\CME Course\\Rew_Property_Data\\Scraped_Data\\Vancouver"

# Initialize the WebDriver
driver = webdriver.Chrome()  

# Use the provided URL for Toronto real estate listings
url = "https://www.rew.ca/properties/areas/vancouver-bc/page/25"
driver.get(url)
print("Waiting 50 seconds for the page to load fully...")
time.sleep(50)

# Loop through a maximum of 20 pages
for page in range(1, 25):
    # Wait for the page to load and the listings to be present
    WebDriverWait(driver, 300).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.displaypanel-body'))
    )

    # Prepare a list to hold all listings data for the current page
    listings_data = []

    # Find listings using the updated CSS selector
    listings = driver.find_elements(By.CSS_SELECTOR, 'div.displaypanel-body')

    # Loop through listings to extract data
    for listing in listings:
        try:
            # Extract price
            price = listing.find_element(By.CSS_SELECTOR, 'div.displaypanel-price').text

            # Extract address
            address = listing.find_element(By.CSS_SELECTOR, 'div.displaypanel-address').text

            # Extract details (e.g., bedrooms, bathrooms, size)
            details = listing.find_elements(By.CSS_SELECTOR, 'ul.inlinelist li')
            bedrooms = details[0].text if len(details) > 0 else 'N/A'
            bathrooms = details[1].text if len(details) > 1 else 'N/A'
            size = details[2].text if len(details) > 2 else 'N/A'

            # Extract home type (if available)
            home_type_element = listing.find_elements(By.CSS_SELECTOR, 'div.displaypanel-info')
            home_type = home_type_element[0].text if home_type_element else 'N/A'

            # Append the extracted data to the listings_data list
            listings_data.append([address, price, bedrooms, bathrooms, size, home_type])
        except Exception as e:
            print(f"Error extracting data from a listing: {e}")

    # Define the output file for the current page
    output_file = os.path.join(base_dir, f"rew_listings_Vancouver_Page_{page}.csv")

    # Write the data for the current page to a CSV file
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(['Address', 'Price', 'Bedrooms', 'Bathrooms', 'Size', 'Home Type'])
        # Write the data rows
        writer.writerows(listings_data)

    print(f"Data for page {page} saved to {output_file}")

    # Attempt to find the "Next" button and navigate to the next page
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[rel="next"]')
        next_page_url = next_button.get_attribute('href')
        driver.get(next_page_url)
        time.sleep(50)  # Allow the next page to load
    except Exception as e:
        print(f"Error navigating to the next page: {e}")
        break

# Close the driver
driver.quit()
print("Scraping completed.")